
🔐 GITHUB PUSH - ENTER YOUR NEW TOKEN
If you haven't created a token yet:
1. Go to: https://github.com/settings/tokens/new
2. Create token with 'repo' permissions
3. Copy the token and paste it below
Paste your NEW GitHub Personal Access Token: ··········
🚀 Pushing to GitHub...
/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `git push https://mudit-mohit:# QUICK PUSH TO FIX GITHUB NOTEBOOK - Run this entire block from google.colab import drive import os import shutil import getpass  print("🔄 Fixing GitHub notebook issue...")  # Mount Google Drive drive.mount('/content/drive')  # Clone your existing repository to get the other files print("📥 Cloning your existing repository...") !git clone https://github.com/mudit-mohit/shakespeare-text-generator.git %cd shakespeare-text-generator  # Copy your working notebook from Colab Notebooks colab_notebooks_path = "/content/drive/MyDrive/Colab Notebooks" notebook_name = "LSTM Text Generator.ipynb" notebook_path

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import requests
import re
from sklearn.model_selection import train_test_split
import random

In [8]:
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


In [9]:
class ShakespeareTextGenerator:
    def __init__(self, sequence_length=40, step=3):
        self.sequence_length = sequence_length
        self.step = step
        self.model = None
        self.char_to_idx = {}
        self.idx_to_char = {}

    def download_shakespeare_data(self):
        """Download Shakespeare's complete works from Project Gutenberg"""
        print("Downloading Complete Shakespeare Works from Project Gutenberg...")
        url = "https://www.gutenberg.org/cache/epub/100/pg100.txt"

        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()

            text = response.text

            # Remove Gutenberg header and footer
            start_pattern = r"\*\*\* START OF THE PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE \*\*\*"
            end_pattern = r"\*\*\* END OF THE PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE \*\*\*"

            start_match = re.search(start_pattern, text, re.IGNORECASE)
            end_match = re.search(end_pattern, text, re.IGNORECASE)

            if start_match and end_match:
                text = text[start_match.end():end_match.start()]

            self.text = text
            print(f"Downloaded Complete Shakespeare: {len(self.text):,} characters")
            print(f"First 300 characters:\n{self.text[:300]}...\n")

        except Exception as e:
            print(f"Download failed: {e}")
            print("Using substantial fallback text...")
            self._create_fallback_text()

    def _create_fallback_text(self):
        """Create a substantial fallback text for testing"""
        sample_plays = [
            "To be, or not to be: that is the question: Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune, Or to take arms against a sea of troubles, And by opposing end them?",
            "Romeo, Romeo, wherefore art thou Romeo? Deny thy father and refuse thy name; Or, if thou wilt not, be but sworn my love, And I'll no longer be a Capulet.",
            "All the world's a stage, And all the men and women merely players: They have their exits and their entrances; And one man in his time plays many parts.",
            "Now is the winter of our discontent Made glorious summer by this sun of York; And all the clouds that lour'd upon our house In the deep bosom of the ocean buried.",
            "Shall I compare thee to a summer's day? Thou art more lovely and more temperate: Rough winds do shake the darling buds of May, And summer's lease hath all too short a date:"
        ]
        self.text = " \n".join(sample_plays * 80)
        print(f"Created fallback text: {len(self.text):,} characters")

    def preprocess_text(self):
        """Preprocess the text: lowercase and clean"""
        print("Preprocessing text...")

        # Convert to lowercase
        text = self.text.lower()

        # Keep basic punctuation and clean up
        text = re.sub(r'\n+', '\n', text)
        text = re.sub(r'[ ]+', ' ', text)

        # Get all unique characters
        self.chars = sorted(list(set(text)))
        self.vocab_size = len(self.chars)

        print(f"Total characters: {len(text):,}")
        print(f"Unique characters: {self.vocab_size}")
        print(f"Character set sample: {''.join(self.chars[:30])}...")

        # Create character to index mapping
        self.char_to_idx = {char: idx for idx, char in enumerate(self.chars)}
        self.idx_to_char = {idx: char for idx, char in enumerate(self.chars)}

        return text

    def create_sequences(self, text):
        """Create input-output sequences for training"""
        print("Creating sequences...")

        # Use substantial portion of text for training
        training_text = text[:400000]
        print(f"Using {len(training_text):,} characters for sequence creation")

        sentences = []
        next_chars = []

        for i in range(0, len(training_text) - self.sequence_length, self.step):
            sentences.append(training_text[i:i + self.sequence_length])
            next_chars.append(training_text[i + self.sequence_length])

        print(f"Created {len(sentences):,} sequences")

        # Convert to numerical data
        X = np.zeros((len(sentences), self.sequence_length), dtype=np.int32)
        y = np.zeros((len(sentences), self.vocab_size), dtype=np.bool_)

        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                X[i, t] = self.char_to_idx[char]
            y[i, self.char_to_idx[next_chars[i]]] = 1

        return X, y

    def build_model(self, embedding_dim=50, lstm_units=128):
        """Build the basic LSTM model architecture"""
        print("Building basic LSTM model...")

        self.model = Sequential([
            Embedding(self.vocab_size, embedding_dim, input_length=self.sequence_length),
            LSTM(lstm_units, return_sequences=True),
            Dropout(0.2),
            LSTM(lstm_units),
            Dropout(0.2),
            Dense(self.vocab_size, activation='softmax')
        ])

        self.model.compile(
            loss='categorical_crossentropy',
            optimizer=Adam(learning_rate=0.001),
            metrics=['accuracy']
        )

        print(self.model.summary())
        return self.model

    def train_model(self, X, y, epochs=30, batch_size=128):
        """Train the LSTM model"""
        print("Training model...")

        # Split data
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        print(f"Training on {len(X_train):,} sequences")
        print(f"Validating on {len(X_val):,} sequences")

        # Callbacks
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
            ModelCheckpoint('basic_shakespeare_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
        ]

        # Train
        history = self.model.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            verbose=1
        )

        return history

    def sample(self, preds, temperature=1.0):
        """Sample from probability distribution with temperature"""
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)

    def generate_text(self, seed_text, length=400, temperature=0.7):
        """Generate new text using the trained model"""
        if self.model is None:
            raise ValueError("Model must be trained before generating text")

        print(f"Generating text with seed: '{seed_text}'...")
        generated = seed_text.lower()

        # Prepare seed
        if len(seed_text) < self.sequence_length:
            seed_text = " " * (self.sequence_length - len(seed_text)) + seed_text
        else:
            seed_text = seed_text[-self.sequence_length:]

        for i in range(length):
            # Convert to numerical
            x = np.zeros((1, self.sequence_length))
            for t, char in enumerate(seed_text):
                x[0, t] = self.char_to_idx.get(char, self.char_to_idx.get(' ', 0))

            # Predict next character
            preds = self.model.predict(x, verbose=0)[0]
            next_index = self.sample(preds, temperature)
            next_char = self.idx_to_char[next_index]

            # Update
            generated += next_char
            seed_text = seed_text[1:] + next_char

            # Show progress for long generations
            if i % 100 == 0:
                print(f"Progress: {i}/{length} characters generated...")

        return self.format_generated_text(generated)

    def format_generated_text(self, text):
        """Format the generated text"""
        # Basic capitalization and formatting
        sentences = text.split('. ')
        sentences = [s.strip().capitalize() for s in sentences if s.strip()]
        text = '. '.join(sentences)

        # Add line breaks
        text = text.replace('. ', '.\n')
        text = text.replace('? ', '?\n')
        text = text.replace('! ', '!\n')

        return text

In [10]:
class AdvancedShakespeareGenerator:
    def __init__(self, sequence_length=50, step=3):
        self.sequence_length = sequence_length
        self.step = step
        self.model = None
        self.char_to_idx = {}
        self.idx_to_char = {}

    def download_shakespeare_data(self):
        """Download the same Project Gutenberg dataset"""
        print("Downloading Complete Shakespeare Works for Advanced Model...")
        url = "https://www.gutenberg.org/cache/epub/100/pg100.txt"

        try:
            response = requests.get(url, timeout=30)
            response.raise_for_status()

            text = response.text

            # Remove Gutenberg header and footer
            start_pattern = r"\*\*\* START OF THE PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE \*\*\*"
            end_pattern = r"\*\*\* END OF THE PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE \*\*\*"

            start_match = re.search(start_pattern, text, re.IGNORECASE)
            end_match = re.search(end_pattern, text, re.IGNORECASE)

            if start_match and end_match:
                text = text[start_match.end():end_match.start()]

            self.text = text
            print(f"Downloaded Complete Shakespeare: {len(self.text):,} characters")

        except Exception as e:
            print(f"Download failed: {e}")
            print("Using substantial fallback text for advanced model...")
            self._create_fallback_text()

    def _create_fallback_text(self):
        """Create substantial fallback text"""
        sample_plays = [
            "To be, or not to be: that is the question: Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune, Or to take arms against a sea of troubles, And by opposing end them?",
            "Romeo, Romeo, wherefore art thou Romeo? Deny thy father and refuse thy name; Or, if thou wilt not, be but sworn my love, And I'll no longer be a Capulet.",
            "All the world's a stage, And all the men and women merely players: They have their exits and their entrances; And one man in his time plays many parts.",
            "Now is the winter of our discontent Made glorious summer by this sun of York; And all the clouds that lour'd upon our house In the deep bosom of the ocean buried.",
            "Shall I compare thee to a summer's day? Thou art more lovely and more temperate: Rough winds do shake the darling buds of May, And summer's lease hath all too short a date:"
        ]
        self.text = " \n".join(sample_plays * 100)
        print(f"Created advanced fallback text: {len(self.text):,} characters")

    def preprocess_text(self):
        """Advanced preprocessing"""
        print("Advanced preprocessing...")

        text = self.text.lower()
        text = re.sub(r'\n+', '\n', text)
        text = re.sub(r'[ ]+', ' ', text)

        self.chars = sorted(list(set(text)))
        self.vocab_size = len(self.chars)

        print(f"Total characters: {len(text):,}")
        print(f"Unique characters: {self.vocab_size}")
        print(f"Character set size: {len(self.chars)}")

        self.char_to_idx = {char: idx for idx, char in enumerate(self.chars)}
        self.idx_to_char = {idx: char for idx, char in enumerate(self.chars)}

        return text

    def create_sequences(self, text):
        """Create sequences with larger context"""
        print("Creating sequences for advanced model...")

        training_text = text[:600000]
        print(f"Using {len(training_text):,} characters for advanced training")

        sentences = []
        next_chars = []

        for i in range(0, len(training_text) - self.sequence_length, self.step):
            sentences.append(training_text[i:i + self.sequence_length])
            next_chars.append(training_text[i + self.sequence_length])

        print(f"Created {len(sentences):,} sequences for advanced training")

        X = np.zeros((len(sentences), self.sequence_length), dtype=np.int32)
        y = np.zeros((len(sentences), self.vocab_size), dtype=np.bool_)

        for i, sentence in enumerate(sentences):
            for t, char in enumerate(sentence):
                X[i, t] = self.char_to_idx[char]
            y[i, self.char_to_idx[next_chars[i]]] = 1

        return X, y

    def build_advanced_model(self, embedding_dim=100, lstm_units=256):
        """Build advanced model with bidirectional LSTM and deeper architecture"""
        print("Building advanced LSTM model...")

        self.model = Sequential([
            Embedding(self.vocab_size, embedding_dim, input_length=self.sequence_length),
            Bidirectional(LSTM(lstm_units, return_sequences=True)),
            Dropout(0.3),
            LSTM(lstm_units, return_sequences=True),
            Dropout(0.3),
            LSTM(lstm_units // 2),
            Dropout(0.3),
            Dense(self.vocab_size * 2, activation='relu'),
            Dropout(0.2),
            Dense(self.vocab_size, activation='softmax')
        ])

        self.model.compile(
            loss='categorical_crossentropy',
            optimizer=Adam(learning_rate=0.001),
            metrics=['accuracy']
        )

        print(self.model.summary())
        return self.model

    def train_advanced_model(self, X, y, epochs=35, batch_size=256):
        """Advanced training with more callbacks"""
        print("Training advanced model...")

        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.1, random_state=42
        )

        print(f"Training on {len(X_train):,} sequences")
        print(f"Validating on {len(X_val):,} sequences")

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1),
            ModelCheckpoint('advanced_shakespeare_model.h5', monitor='val_loss', save_best_only=True, verbose=1),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)
        ]

        history = self.model.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks,
            verbose=1
        )

        return history

    def sample(self, preds, temperature=1.0):
        """Advanced sampling with better numerical stability"""
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds + 1e-8) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)

    def generate_advanced_text(self, seed_text, length=500, temperature=0.7):
        """Generate text with advanced model"""
        if self.model is None:
            raise ValueError("Advanced model must be trained first")

        print(f"Advanced generation with seed: '{seed_text}'...")
        generated = seed_text.lower()

        if len(seed_text) < self.sequence_length:
            seed_text = " " * (self.sequence_length - len(seed_text)) + seed_text
        else:
            seed_text = seed_text[-self.sequence_length:]

        for i in range(length):
            x = np.zeros((1, self.sequence_length))
            for t, char in enumerate(seed_text):
                x[0, t] = self.char_to_idx.get(char, self.char_to_idx.get(' ', 0))

            preds = self.model.predict(x, verbose=0)[0]
            next_index = self.sample(preds, temperature)
            next_char = self.idx_to_char[next_index]

            generated += next_char
            seed_text = seed_text[1:] + next_char

            # Show progress
            if i % 100 == 0:
                print(f"Progress: {i}/{length} characters generated...")

        return self.format_advanced_text(generated)

    def format_advanced_text(self, text):
        """Better formatting for advanced generation"""
        text = text.capitalize()
        text = re.sub(r'([.!?])', r'\1 ', text)
        text = re.sub(r'[ ]+', ' ', text)

        # Add line breaks for readability
        lines = []
        current_line = ""
        for word in text.split():
            if len(current_line + word) < 80:
                current_line += word + " "
            else:
                lines.append(current_line)
                current_line = word + " "
        lines.append(current_line)

        return '\n'.join(lines)

In [11]:
def run_basic_generator():
    """Run the basic Shakespeare text generator"""
    print("=" * 70)
    print("BASIC SHAKESPEARE TEXT GENERATOR")
    print("USING COMPLETE PROJECT GUTENBERG DATASET")
    print("=" * 70)

    # Initialize and setup
    generator = ShakespeareTextGenerator(sequence_length=40, step=3)
    generator.download_shakespeare_data()
    processed_text = generator.preprocess_text()

    # Create sequences and train
    X, y = generator.create_sequences(processed_text)
    generator.build_model(embedding_dim=50, lstm_units=128)

    print("\nTraining Basic Model...")
    history = generator.train_model(X, y, epochs=25, batch_size=128)

    # Generate examples
    print("\n" + "=" * 50)
    print("BASIC MODEL GENERATED TEXT EXAMPLES")
    print("=" * 50)

    seeds = [
        "to be or not to be",
        "romeo romeo wherefore art thou",
        "shall i compare thee to a summer",
        "now is the winter of our discontent"
    ]

    for seed in seeds:
        print(f"\nSeed: '{seed}'")
        print("-" * 50)
        for temp in [0.3, 0.7, 1.0]:
            print(f"\nTemperature {temp}:")
            generated_text = generator.generate_text(
                seed_text=seed,
                length=300,
                temperature=temp
            )
            print(generated_text)
            print("\n" + "-" * 40)

In [12]:
def run_advanced_generator():
    """Run the advanced Shakespeare text generator"""
    print("\n" + "=" * 70)
    print("ADVANCED SHAKESPEARE TEXT GENERATOR")
    print("USING COMPLETE PROJECT GUTENBERG DATASET")
    print("=" + "=" * 69)

    # Initialize advanced generator
    advanced_generator = AdvancedShakespeareGenerator(sequence_length=50, step=3)
    advanced_generator.download_shakespeare_data()
    processed_text = advanced_generator.preprocess_text()

    # Create sequences and train advanced model
    X, y = advanced_generator.create_sequences(processed_text)
    advanced_generator.build_advanced_model(embedding_dim=100, lstm_units=256)

    print("\nTraining Advanced Model...")
    history = advanced_generator.train_advanced_model(X, y, epochs=30, batch_size=256)

    # Generate advanced examples
    print("\n" + "=" * 50)
    print("ADVANCED MODEL GENERATED TEXT EXAMPLES")
    print("=" * 50)

    advanced_seeds = [
        "to be or not to be that is the question",
        "now is the winter of our discontent made glorious",
        "shall i compare thee to a summer's day thou art",
        "romeo, romeo, wherefore art thou romeo deny thy",
        "all the world's a stage and all the men and women"
    ]

    for seed in advanced_seeds:
        print(f"\nAdvanced Seed: '{seed}'")
        print("-" * 60)
        for temp in [0.3, 0.7, 1.2]:
            print(f"\nTemperature {temp}:")
            advanced_text = advanced_generator.generate_advanced_text(
                seed_text=seed,
                length=400,
                temperature=temp
            )
            print(advanced_text)
            print("\n" + "─" * 50)

In [13]:
def compare_architectures():
    """Compare basic vs advanced architectures"""
    print("\n" + "=" * 70)
    print("ARCHITECTURE COMPARISON: BASIC vs ADVANCED")
    print("=" * 70)

    print("\nBASIC ARCHITECTURE:")
    print("   • Single direction LSTM")
    print("   • 2 LSTM layers (128 units each)")
    print("   • Basic dropout (0.2)")
    print("   • Simple embedding (50 dim)")
    print("   • Faster training, less memory")
    print("   • Good for quick results")

    print("\nADVANCED ARCHITECTURE:")
    print("   • Bidirectional LSTM")
    print("   • 3 LSTM layers (256, 256, 128 units)")
    print("   • Advanced dropout (0.3)")
    print("   • Larger embedding (100 dim)")
    print("   • Additional dense layer")
    print("   • Learning rate scheduling")
    print("   • Better context understanding")
    print("   • Richer vocabulary and style")

In [16]:
if __name__ == "__main__":
    print("SHAKESPEARE TEXT GENERATOR")
    print("Complete Project Gutenberg Dataset")
    print("Basic and Advanced LSTM Models")
    print("=" * 70)

    # Ask user which model to run
    print("\nChoose which model to run:")
    print("1. Basic Model Only (Faster)")
    print("2. Advanced Model Only (Better Quality)")
    print("3. Both Models (Complete Comparison)")

    try:
        choice = input("\nEnter your choice (1/2/3): ").strip()

        if choice == "1":
            print("\n" + "🚀" * 20)
            print("RUNNING BASIC MODEL...")
            print("🚀" * 20)
            run_basic_generator()

        elif choice == "2":
            print("\n" + "🚀" * 20)
            print("RUNNING ADVANCED MODEL...")
            print("🚀" * 20)
            run_advanced_generator()

        elif choice == "3":
            print("\n" + "🚀" * 20)
            print("RUNNING BOTH MODELS - COMPLETE COMPARISON")
            print("🚀" * 20)
            run_basic_generator()
            run_advanced_generator()
            compare_architectures()

        else:
            print("Invalid choice. Running both models by default...")
            run_basic_generator()
            run_advanced_generator()
            compare_architectures()

    except KeyboardInterrupt:
        print("\nExecution interrupted by user")
    except Exception as e:
        print(f"\nError during execution: {e}")
        print("Trying to run basic model only...")
        run_basic_generator()

    print("\n" + "🎉" * 20)
    print("SCRIPT EXECUTION COMPLETED!")
    print("🎉" * 20)
    print("\nModel files saved:")
    print("   • basic_shakespeare_model.h5")
    print("   • advanced_shakespeare_model.h5")
    print("\nHappy Shakespearean text generating!")

SHAKESPEARE TEXT GENERATOR
Complete Project Gutenberg Dataset
Basic and Advanced LSTM Models

Choose which model to run:
1. Basic Model Only (Faster)
2. Advanced Model Only (Better Quality)
3. Both Models (Complete Comparison)

🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
RUNNING BOTH MODELS - COMPLETE COMPARISON
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
BASIC SHAKESPEARE TEXT GENERATOR
USING COMPLETE PROJECT GUTENBERG DATASET
Downloaded Complete Shakespeare: 5,555,374 characters
First 300 characters:





The Complete Works of William Shakespeare

by William Shakespeare




                    Contents

    THE SONNETS
    ALL’S WELL THAT ENDS WELL
    THE TRAGEDY OF ANTONY AND CLEOPATRA
    AS YOU LIKE IT
    THE COMEDY OF ERRORS
    THE TRAGEDY OF CORIOLANUS
    CYMBELINE
...

Preprocessing text...
Total characters: 5,541,615
Unique characters: 71
Character set sample: 	
 !&'()*,-.0123456789:;?[]_a...
Creating sequences...
Using 400,000 characters for sequence creation
✅ Created 133,320 sequences
Building basic LSTM model...


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

704/704 ━━━━━━━━━━━━━━━━━━━━ 45s 63ms/step - accuracy: 0.5333 - loss: 1.5172 - val_accuracy: 0.5405 - val_loss: 1.5017 - learning_rate: 0.0010
Epoch 19/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5371 - loss: 1.5010
Epoch 19: val_loss improved from 1.50169 to 1.49284, saving model to advanced_shakespeare_model.h5


704/704 ━━━━━━━━━━━━━━━━━━━━ 45s 64ms/step - accuracy: 0.5371 - loss: 1.5010 - val_accuracy: 0.5442 - val_loss: 1.4928 - learning_rate: 0.0010
Epoch 20/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5384 - loss: 1.4986
Epoch 20: val_loss did not improve from 1.49284
704/704 ━━━━━━━━━━━━━━━━━━━━ 45s 63ms/step - accuracy: 0.5384 - loss: 1.4986 - val_accuracy: 0.5461 - val_loss: 1.4948 - learning_rate: 0.0010
Epoch 21/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5435 - loss: 1.4810
Epoch 21: val_loss improved from 1.49284 to 1.48397, saving model to advanced_shakespeare_model.h5


704/704 ━━━━━━━━━━━━━━━━━━━━ 45s 64ms/step - accuracy: 0.5435 - loss: 1.4810 - val_accuracy: 0.5467 - val_loss: 1.4840 - learning_rate: 0.0010
Epoch 22/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5447 - loss: 1.4693
Epoch 22: val_loss did not improve from 1.48397
704/704 ━━━━━━━━━━━━━━━━━━━━ 44s 63ms/step - accuracy: 0.5447 - loss: 1.4693 - val_accuracy: 0.5457 - val_loss: 1.4893 - learning_rate: 0.0010
Epoch 23/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5491 - loss: 1.4625
Epoch 23: val_loss improved from 1.48397 to 1.47977, saving model to advanced_shakespeare_model.h5


704/704 ━━━━━━━━━━━━━━━━━━━━ 45s 64ms/step - accuracy: 0.5491 - loss: 1.4624 - val_accuracy: 0.5517 - val_loss: 1.4798 - learning_rate: 0.0010
Epoch 24/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5533 - loss: 1.4443
Epoch 24: val_loss improved from 1.47977 to 1.47231, saving model to advanced_shakespeare_model.h5


704/704 ━━━━━━━━━━━━━━━━━━━━ 45s 63ms/step - accuracy: 0.5533 - loss: 1.4443 - val_accuracy: 0.5512 - val_loss: 1.4723 - learning_rate: 0.0010
Epoch 25/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5535 - loss: 1.4409
Epoch 25: val_loss did not improve from 1.47231
704/704 ━━━━━━━━━━━━━━━━━━━━ 45s 63ms/step - accuracy: 0.5535 - loss: 1.4409 - val_accuracy: 0.5545 - val_loss: 1.4742 - learning_rate: 0.0010
Epoch 26/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5593 - loss: 1.4230
Epoch 26: val_loss did not improve from 1.47231
704/704 ━━━━━━━━━━━━━━━━━━━━ 45s 63ms/step - accuracy: 0.5593 - loss: 1.4230 - val_accuracy: 0.5521 - val_loss: 1.4796 - learning_rate: 0.0010
Epoch 27/30
703/704 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5572 - loss: 1.4187
Epoch 27: val_loss did not improve from 1.47231

Epoch 27: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
704/704 ━━━━━━━━━━━━━━━━━━━━ 45s 63ms/step - accuracy: 0.5572 - loss: 1.4187 - val_ac